TODO:
1. Найти распределение вероятностей для каждого состояния
2. Построить график количества людей в системе от времени
3. Построить график среднего времени нахождения клиента в системе от времени

In [1]:
import threading
import random
import time

In [19]:
N_CLIENTS = 20

LAMBDA = 1
MU1 = 1
MU2 = 0.5

LOGGING = True

clients_list = []

event = threading.Event()
event.set()

In [3]:
class Client:
    def __init__(self, id):
        self.id = id
        self.start_time = None
        self.finish_time = None


In [4]:
class Chair1:
    def __init__(self):
        self.is_busy = False
    
    def serving(self, client: Client):
        random.seed()
        ksi = random.expovariate(MU1)
        self.is_busy = True
        if LOGGING:
            print(f'client {client.id} is starting serving on the 1 chair')
        time.sleep(ksi)
        if LOGGING:
            print(f'client {client.id} finished serving on the 1 chair')
    

In [5]:
class Chair2:
    def __init__(self):
        self.is_busy = False
    
    def serving(self, client: Client):
        random.seed()
        ksi = random.expovariate(MU2)
        self.is_busy = True
        if LOGGING:
            print(f'client {client.id} is starting serving on the 2 chair')
        time.sleep(ksi)
        if LOGGING:
            print(f'client {client.id} finished serving on the 2 chair')

In [20]:
def serving(client: Client, chair1: Chair1, chair2: Chair2):
    if LOGGING:
        print(f'CLIENT {client.id} STARTED SERVING')
    client.start_time = time.time()
    chair1.serving(client)
    if not event.is_set():
        if LOGGING:
            print(f'client {client.id} is waiting for chair 2')
    event.wait()
    event.clear()
    chair1.is_busy = False
    chair2.serving(client)
    event.set()
    client.finish_time = time.time()
    clients_list.append(client)
    if LOGGING:
        print(f'CLIENT {client.id} FINISHED SERVING')

In [22]:
chair1 = Chair1()
chair2 = Chair2()

serving_start = time.time()

for i in range(N_CLIENTS):
    random.seed()
    ksi = random.expovariate(LAMBDA)
    time.sleep(ksi)
    if not chair1.is_busy:
        x = threading.Thread(target=serving, args=(Client(i + 1), chair1, chair2))
        x.start()
    else:
        if LOGGING:
            print(f'CLIENT {i + 1} WAS REJECTED')
    
x.join()

serving_end = time.time()

total_time = serving_end - serving_start

if LOGGING:
    print(f'Serving finished, total time: {total_time}')
    
for client in clients_list:
    print(f'id: {client.id}, time in system: {client.finish_time - client.start_time}')

CLIENT 1 STARTED SERVING
client 1 is starting serving on the 1 chair
client 1 finished serving on the 1 chair
client 1 is starting serving on the 2 chair
client 1 finished serving on the 2 chair
CLIENT 1 FINISHED SERVING
CLIENT 2 STARTED SERVING
client 2 is starting serving on the 1 chair
client 2 finished serving on the 1 chair
client 2 is starting serving on the 2 chair
CLIENT 3 STARTED SERVING
client 3 is starting serving on the 1 chair
client 3 finished serving on the 1 chair
client 3 is waiting for chair 2
client 2 finished serving on the 2 chair
CLIENT 2 FINISHED SERVING
client 3 is starting serving on the 2 chair
CLIENT 4 STARTED SERVING
client 4 is starting serving on the 1 chair
CLIENT 5 WAS REJECTED
CLIENT 6 WAS REJECTED
CLIENT 7 WAS REJECTED
CLIENT 8 WAS REJECTED
client 4 finished serving on the 1 chair
client 4 is waiting for chair 2
CLIENT 9 WAS REJECTED
CLIENT 10 WAS REJECTED
client 3 finished serving on the 2 chair
CLIENT 3 FINISHED SERVING
client 4 is starting serving o